In [ ]:
!pip install ultralytics
!pip install roboflow
!pip install supervision

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="LEmZLocu5LCHOkUvVoOS")
project = rf.workspace("ashwin-bhai-ki-help").project("tactile_mapcrop_dataset")
version = project.version(3)
dataset = version.download("yolov8-obb")

In [ ]:
import yaml
import os

In [ ]:
# Define the path to your data.yaml file
data_yaml_path = os.path.join(dataset.location, '/kaggle/working/TacTile_mapCrop_Dataset-3/data.yaml')

# Check if data.yaml exists
if not os.path.exists(data_yaml_path):
    print(f"Error: data.yaml not found at {data_yaml_path}")
else:
    # Read the data.yaml file
    with open(data_yaml_path, 'r') as file:
        data = yaml.safe_load(file)

    # Update the 'path' key to reflect the current dataset location
    # This is often necessary in Kaggle as the initial path might be absolute or incorrect.
    data['path'] = dataset.location

    # Ensure train, val, test paths are relative to the 'path'
    # Roboflow usually exports them correctly, but it's good to confirm.
    # Adjust these if your folder structure is different within the downloaded dataset.
    data['train'] = 'train/images'
    data['val'] = 'valid/images'
    # If you have a test set and want to use it, uncomment the line below:
    # data['test'] = 'test/images'

    # Write the updated data.yaml back to the file
    with open(data_yaml_path, 'w') as file:
        yaml.dump(data, file, sort_keys=False)

    print(f"Updated data.yaml content:\n{data}")

# You can also list the contents of your downloaded dataset directory to verify
print("\nContents of downloaded dataset directory:")
!ls -R {dataset.location}

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 OBB model.
# 'yolov8n-obb.pt' is the nano version, good for quick experiments.
# You can choose 'yolov8s-obb.pt', 'yolov8m-obb.pt', 'yolov8l-obb.pt', or 'yolov8x-obb.pt'
# depending on your desired trade-off between speed and accuracy, and available GPU memory.
model = YOLO('yolov8n-obb.pt')

print("YOLOv8 OBB model loaded successfully.")

In [ ]:
# Assuming you have already executed the following:
# !pip install ultralytics
# !pip install supervision
# from roboflow import Roboflow
# rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY") # Replace with your actual API key
# project = rf.workspace("ashwin-bhai-ki-help").project("tactile_mapcrop_dataset")
# version = project.version(3)
# dataset = version.download("yolov8-obb")
# And the data.yaml verification/update part...
# from ultralytics import YOLO
# model = YOLO('yolov8n-obb.pt')


# --- Define training parameters ---
epochs = 300       # Set a sufficiently high number of epochs. Early stopping will halt it.
imgsz = 640        # Image size for training (square images are common)
batch_size = 16    # Batch size (adjust based on your GPU memory; smaller if OOM)
project_name = 'YOLOv8_OBB_Training' # Name for your training project
# It's good practice to change the run name when you change parameters
run_name = 'tactile_map_obb_run_with_early_stopping' 

# --- Early Stopping Parameter ---
# Training will stop if the validation metric (mAP50-95 for OBB) does not improve
# for 'patience_epochs' consecutive epochs.
patience_epochs = 50 # Example: Stop if no improvement for 50 consecutive epochs

print(f"Starting training with data from: {data_yaml_path}")
print(f"Early stopping patience set to: {patience_epochs} epochs")

# --- Train the model with early stopping ---
results = model.train(
    data=data_yaml_path,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    project=project_name,
    name=run_name,
    patience=patience_epochs, # <--- THIS IS WHERE EARLY STOPPING IS APPLIED
    # You can add other parameters here, e.g.,
    # lr0=0.01, # Initial learning rate
    # optimizer='AdamW', # Choose optimizer
    # device=0 # Use GPU 0, or 'cpu' for CPU training (much slower)
)

print("\nTraining complete!")

# --- Post-Training Evaluation and Prediction (remains largely the same) ---
import os
import cv2
import supervision as sv
import random # For picking a random test image
import matplotlib.pyplot as plt # Make sure matplotlib is imported if you want to display images

# Path to your best trained model weights
# The path will be something like: /kaggle/working/YOLOv8_OBB_Training/tactile_map_obb_run_with_early_stopping/weights/best.pt
best_model_path = os.path.join('/kaggle/working', project_name, run_name, 'weights', 'best.pt')

if os.path.exists(best_model_path):
    print(f"\nLoading best model from: {best_model_path}")
    model = YOLO(best_model_path)

    # Validate the model on the validation set
    print("\nRunning validation on the trained model:")
    metrics = model.val(data=data_yaml_path)

    print(f"mAP50-95(B): {metrics.box.map}")
    print(f"mAP50(B): {metrics.box.map50}")
    print(f"mAP(B) per class: {metrics.box.maps}")

    # --- Make predictions on a sample image ---
    print("\nMaking predictions on a sample test image:")
    test_images_dir = os.path.join(dataset.location, 'test', 'images')

    if os.path.exists(test_images_dir) and os.listdir(test_images_dir):
        random_image_name = random.choice(os.listdir(test_images_dir))
        image_to_predict_path = os.path.join(test_images_dir, random_image_name)

        print(f"Predicting on: {image_to_predict_path}")

        # Perform inference
        inference_results = model(image_to_predict_path)

        # Get the first result object (assuming batch size 1 for simplicity here)
        result = inference_results[0]

        # Extract OBB detections using supervision
        detections = sv.Detections.from_ultralytics(result)

        # Annotate the image
        image = cv2.imread(image_to_predict_path)
        if image is None:
            print(f"Error: Could not read image at {image_to_predict_path}")
        else:
            oriented_box_annotator = sv.OrientedBoxAnnotator()
            annotated_frame = oriented_box_annotator.annotate(
                scene=image.copy(),
                detections=detections
            )

            # Display the annotated image
            plt.figure(figsize=(10, 10))
            plt.imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.title("Predicted OBBs")
            plt.show()

            # Print raw OBB results
            print("Raw OBB prediction results:")
            print(result.obb)
            print("Class names:", result.names) # Map class IDs to names

    else:
        print("No test images found or directory is empty. Cannot perform sample prediction.")
else:
    print(f"Error: Best model not found at {best_model_path}. Training might not have completed successfully.")

In [ ]:
from ultralytics import YOLO
import os

# Assuming your previous training saved the model in this path
project_name = 'YOLOv8_OBB_Training'
run_name = 'tactile_map_obb_run_with_early_stopping' # Or whatever your last run name was
best_model_path = os.path.join('/kaggle/working', project_name, run_name, 'weights', 'best.pt')

model = YOLO(best_model_path)
print(f"Loaded model from: {best_model_path}")

In [ ]:
# Export to TFLite (float32)
tflite_model_path = model.export(format='tflite', imgsz=640) # imgsz should match your training size
print(f"Model exported to: {tflite_model_path}")

# Optional: Export with INT8 quantization (reduces size significantly, may affect accuracy)
# This requires a representative dataset for calibration.
# If your dataset has a 'test' split, you can use it for calibration.
# Note: Quantization can be tricky with OBB and might require more specific steps or checking Ultralytics/TensorFlow Lite documentation for full support.
# If the simple 'tflite' export works, start with that.
# You would need to provide a DataLoader or representative dataset for `data` argument here.
# quantized_tflite_model_path = model.export(format='tflite', imgsz=640, int8=True, data=data_yaml_path)
# print(f"Quantized model exported to: {quantized_tflite_model_path}")

# After running this, download the .tflite file from your Kaggle output.
# It will be in the same directory as your best.pt (e.g., /kaggle/working/YOLOv8_OBB_Training/tactile_map_obb_run_with_early_stopping/)
# The file name will be something like 'best_float32.tflite' or 'best_int8.tflite'.

In [ ]:
import os
from IPython.display import FileLink

# Define the name for your zip file
zip_filename = "kaggle_working_output.zip"

# Change to the working directory. This ensures the zip command is run
# from the correct location, making it easier to manage paths within the zip.
os.chdir('/kaggle/working/')

print(f"Current directory: {os.getcwd()}")
print(f"Zipping contents of /kaggle/working/ into {zip_filename}...")

# Use the 'zip' command to compress the entire /kaggle/working/ directory.
# `!`: Executes a shell command
# `zip`: The command-line utility for zipping files
# `-r`: Recursive; includes subdirectories and their contents
# `.`: Zips the current directory (which is /kaggle/working/ because of os.chdir)
# `*`: (Optional, but often useful) Zips all files and directories *directly* within /kaggle/working/
#      If you omit `*`, it will include the /kaggle/working/ parent folder inside the zip.
#      If you include `*`, it will only zip the *contents* of /kaggle/working/.
#      I've used `*` here as it's common to want the contents directly.
!zip -r {zip_filename} ./*

print(f"\nZip file '{zip_filename}' created.")

# Generate a clickable download link for the created zip file.
# The FileLink assumes the file is in the current working directory.
print("Click the link below to download your zipped folder:")
FileLink(f'./{zip_filename}')